In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install soundfile audioread
!pip install tqdm
!pip install imageio[ffmpeg]

In [ ]:
NUM_EPOCHS=1000
LR=0.01
GAMMA=0.7

In [ ]:
!/usr/bin/python -m pip install --upgrade pip

In [ ]:
!pip install pydub

In [ ]:
!pip install --upgrade librosa

IMPORTING REQUIRED LIBRARIES

In [38]:
import os
import json
import torch
import librosa
import numpy as np
from torch.utils.data import Dataset
import os
#import textgrid
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [40]:
device

'cuda'

FOR CONVERTING TEXTGRID TO JSON

In [41]:
# textgrid_dir = "/content/drive/MyDrive/annotation"
# json_dir = "/content/drive/MyDrive/json"
# audio_dir = "/content/drive/MyDrive/Songs for annotation"

# if not os.path.exists(json_dir):
#     os.makedirs(json_dir)

# file_mapping = {}
# counter = 1

# for textgrid_file in os.listdir(textgrid_dir):
#     textgrid_file_path = os.path.join(textgrid_dir, textgrid_file)

#     try:
#         tgrid = textgrid.read_textgrid(textgrid_file_path)


#         new_filename = f"{counter:03d}"
#         json_file_path = os.path.join(json_dir, f"{new_filename}.json")
#         textgrid_new_file_path = os.path.join(textgrid_dir, f"{new_filename}.TextGrid")
#         audio_new_file_path = os.path.join(audio_dir, f"{new_filename}.mp3")


#         counter += 1


#         file_mapping[textgrid_file] = new_filename


#         pd.DataFrame(tgrid).to_json(json_file_path)


#         os.rename(textgrid_file_path, textgrid_new_file_path)


#         audio_original_file_path = os.path.join(audio_dir, textgrid_file.replace(".TextGrid", ".mp3"))
#         os.rename(audio_original_file_path, audio_new_file_path)

#     except Exception as e:
#         pass

# # Save the mapping to a CSV file for reference
# mapping_df = pd.DataFrame(list(file_mapping.items()), columns=['OriginalFilename', 'NewFilename'])
# mapping_df.to_csv("/content/drive/MyDrive/filename_mapping.csv", index=False)


DATASET CLASS AND DATALOADER

In [42]:
import os
import json
from pydub import AudioSegment
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import librosa
import numpy as np

class MusicDataset(Dataset):
    def __init__(self, audio_dir, json_dir):
        self.audio_dir = audio_dir
        self.json_dir = json_dir
        self.audio_files = [file for file in os.listdir(audio_dir) if file.endswith(".mp3")]
        self.json_files = [file for file in os.listdir(json_dir) if file.endswith(".json")]

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file_path = os.path.join(self.audio_dir, self.audio_files[idx])
        
        # Use pydub for audio loading
        audio = AudioSegment.from_mp3(audio_file_path)
        y = np.array(audio.get_array_of_samples(),dtype=float)
        sr = audio.frame_rate

        json_file_path = os.path.join(self.json_dir, self.audio_files[idx].replace(".mp3", ".json"))
        with open(json_file_path, "r") as f:
            json_data = json.load(f)

        # Extract start and stop timings from the JSON data
        start_times = [json_data["start"][str(i)] for i in range(len(json_data["start"]))]
        stop_times = [json_data["stop"][str(i)] for i in range(len(json_data["stop"]))]

        # Pad start and stop times
        start_times = torch.nn.functional.pad(torch.FloatTensor(start_times), (0, 15 - len(start_times)))
        stop_times = torch.nn.functional.pad(torch.FloatTensor(stop_times), (0, 15 - len(stop_times)))

        # Extract MFCCs using librosa
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc = torch.nn.functional.pad(torch.FloatTensor(mfcc), (0, 100 - mfcc.shape[1]))

        # Extract Chroma using librosa
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma = torch.nn.functional.pad(torch.FloatTensor(chroma), (0, 100 - chroma.shape[1]))

        # Extract Tempo using librosa
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
        # Extract Rhythm Patterns (Example: using tempogram) using librosa
        tempogram = librosa.feature.tempogram(onset_envelope=onset_env, sr=sr)
        rhythm_patterns = torch.nn.functional.pad(torch.FloatTensor(tempogram), (0, 100 - tempogram.shape[1]))

        sample = {
            'start_times': start_times,
            'stop_times': stop_times,
            'mfcc': mfcc,
            'chroma': chroma,
            'tempo': torch.FloatTensor([tempo]),  # Tempo as a single value
            'rhythm_patterns': rhythm_patterns
        }

        return sample

# Set your directory paths
audio_directory = "./songs"
json_directory = "./json"

# Create dataset and dataloader
music_dataset = MusicDataset(audio_directory, json_directory)
music_dataloader = DataLoader(music_dataset, batch_size=50, shuffle=True, collate_fn=lambda x: x)


In [43]:
valid_loader = DataLoader(music_dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x)

In [44]:
for i in valid_loader:
    print(i)
    break

[{'start_times': tensor([  0.0000,  25.1465,  97.1913, 121.1918, 157.2372, 169.3034, 196.1156,
        232.2465, 282.2249,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000]), 'stop_times': tensor([ 25.1465,  97.1913, 121.1918, 157.2372, 169.3034, 196.1156, 232.2465,
        282.2249, 288.2732,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000]), 'mfcc': tensor([[504.4515, 504.4515, 504.4515,  ..., 504.4515, 504.4515, 504.4515],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'chroma': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        

MODEL CLASS

In [45]:
# from transformers import HubertModel, HubertConfig

In [46]:
class AudioModel(nn.Module):
    def __init__(self):
        super(AudioModel, self).__init__()

        # Load pre-trained Hubert model
        # config = HubertConfig.from_pretrained("facebook/hubert-large-ll60k")
        # self.audio_encoder = HubertModel.from_pretrained("facebook/hubert-large-ll60k", config=config)

        self.feature_encoder = nn.Sequential(
            nn.Linear(in_features=100, out_features=128),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        self.conv1 = nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv1d(16, 1, kernel_size=3, stride=1, padding=1)

        self.fc_start = nn.Linear(in_features=409, out_features=15)  # Adjusted output size to match ground truth
        self.fc_stop = nn.Linear(in_features=409, out_features=15)   # Adjusted output size to match ground truth

    def forward(self, mfcc, chroma, rhythm_patterns):
        # Combine audio features with other features
        combined_features = torch.cat([mfcc, chroma, rhythm_patterns], dim=1)

        # Encode combined features
        features_encoded = self.feature_encoder(combined_features)

        # Transpose to fit the expected shape for 1D convolution
        features_encoded = features_encoded.permute(0, 2, 1)

        # Apply 1D convolutions
        conv1_output = self.conv1(features_encoded)
        conv2_output = self.conv2(conv1_output)
        conv3_output = self.conv3(conv2_output)
        conv4_output = self.conv4(conv3_output)

        # Squeeze to remove the extra dimension added by the convolution
        conv_output = conv4_output.squeeze(dim=2)

        # Predict start and stop times
        start_times = self.fc_start(conv_output)
        stop_times = self.fc_stop(conv_output)

        return start_times, stop_times

In [47]:
from torch.optim.lr_scheduler import ExponentialLR

In [48]:
model = AudioModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = ExponentialLR(optimizer, gamma=GAMMA)

In [49]:
training_loss_array=[]
valid_loss_array=[]

In [50]:
model.to(device)

AudioModel(
  (feature_encoder): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (conv1): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4): Conv1d(16, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc_start): Linear(in_features=409, out_features=15, bias=True)
  (fc_stop): Linear(in_features=409, out_features=15, bias=True)
)

TRAINING

In [ ]:
for epoch in range(NUM_EPOCHS):
    model.train()
    loss=0.0
    total_loss=0.0
    for batch in tqdm(music_dataloader):
        # audio_data = pad_sequence([sample['audio'] for sample in batch], batch_first=True)
        mfcc = pad_sequence([sample['mfcc'] for sample in batch], batch_first=True)
        chroma = pad_sequence([sample['chroma'] for sample in batch], batch_first=True)
        rhythm_patterns = pad_sequence([sample['rhythm_patterns'] for sample in batch], batch_first=True)
        start_times = pad_sequence([sample['start_times'] for sample in batch], batch_first=True)
        stop_times = pad_sequence([sample['stop_times'] for sample in batch], batch_first=True)

        optimizer.zero_grad()

        start_pred, stop_pred = model(mfcc.to(device), chroma.to(device), rhythm_patterns.to(device))
        start_pred = start_pred.squeeze(dim=1)
        stop_pred = stop_pred.squeeze(dim=1)
        loss_start = criterion(start_pred.to(device), start_times.to(device))
        loss_stop = criterion(stop_pred.to(device), stop_times.to(device))

        total_loss = loss_start + loss_stop
        loss += total_loss.item()
        total_loss.backward()
        optimizer.step()
        scheduler.step()
    avg_training_loss = loss / len(music_dataloader)
    training_loss_array.append(avg_training_loss)
    model.eval()  
    total_valid_loss = 0.0

    with torch.no_grad():  # Disable gradient calculation during validation
        for batch in tqdm(valid_loader):
            mfcc = pad_sequence([sample['mfcc'] for sample in batch], batch_first=True)
            chroma = pad_sequence([sample['chroma'] for sample in batch], batch_first=True)
            rhythm_patterns = pad_sequence([sample['rhythm_patterns'] for sample in batch], batch_first=True)
            start_times = pad_sequence([sample['start_times'] for sample in batch], batch_first=True)
            stop_times = pad_sequence([sample['stop_times'] for sample in batch], batch_first=True)

            optimizer.zero_grad()

            start_pred, stop_pred = model(mfcc.to(device), chroma.to(device), rhythm_patterns.to(device))
            start_pred = start_pred.squeeze(dim=1)
            stop_pred = stop_pred.squeeze(dim=1)
            loss_start = criterion(start_pred.to(device), start_times.to(device))
            loss_stop = criterion(stop_pred.to(device), stop_times.to(device))

            total_valid_loss += (loss_start + loss_stop).item()

    avg_valid_loss = total_valid_loss / len(valid_loader)
    valid_loss_array.append(avg_valid_loss)

    print(f"Epoch {epoch+1} | Training Loss: {avg_training_loss} | Validation Loss: {avg_valid_loss}")
    if epoch%25 == 0:
        torch.save(model,f"Checkpoint:Epoch-{epoch+1}.pth")



In [ ]:
epochs = [i for i in range(1, 54)]  # Epochs from 1 to 53

training_loss = [
    45991.265625, 44109.0703125, 33472.6953125, 33352.515625, 25990.76171875,
    24380.203125, 23231.09375, 22515.69140625, 21980.501953125, 21303.6640625,
    20855.671875, 20493.640625, 19928.1015625, 19772.16015625, 19622.189453125,
    19425.78515625, 19359.923828125, 18570.5, 19108.708984375, 19121.466796875,
    19270.287109375, 19576.7890625, 19225.01953125, 19228.751953125,
    19706.8515625, 19164.138671875, 19812.93359375, 18696.76171875, 19321.0625,
    19023.404296875, 19191.533203125, 19239.54296875, 19265.876953125, 19534.625,
    19259.009765625, 19096.033203125, 19422.9921875, 19276.90234375,
    19080.728515625, 19549.556640625, 19505.57421875, 18923.021484375,
    19383.07421875, 19426.2890625, 18953.04296875, 19432.23046875, 19282.578125,
    19707.91015625, 19424.439453125, 19080.923828125, 19880.771484375,
    18925.181640625, 19818.59375
]

validation_loss = [
    44288.341796875, 33471.9619140625, 32385.208984375, 25410.986328125,
    23953.521484375, 23123.837890625, 22390.798828125, 21395.77734375,
    20538.5146484375, 19858.7685546875, 19689.447265625, 19281.96484375,
    19350.666015625, 19089.0009765625, 19017.884765625, 19032.9375,
    18867.1533203125, 18875.02734375, 18728.6962890625, 18876.05078125,
    18645.6845703125, 18779.103515625, 18853.35546875, 18963.5146484375,
    18714.0927734375, 18692.8818359375, 18796.2060546875, 18721.640625,
    18668.7373046875, 18957.138671875, 18937.251953125, 18729.8037109375,
    18745.0078125, 18862.470703125, 18699.2861328125, 18644.65625,
    18840.8427734375, 18597.466796875, 18785.1865234375, 18760.232421875,
    18637.6591796875, 18544.8935546875, 18701.7451171875, 18799.20703125,
    18857.5146484375, 18658.4208984375, 18744.328125, 18797.5478515625,
    18590.28125, 18825.9189453125, 18536.0166015625, 18749.1279296875,
    18763.5927734375
]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, label='Training Loss', marker='o')
plt.plot(epochs, validation_loss, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('res.png')
plt.show()

In [ ]:
import torch

In [ ]:
model=torch.load('./Checkpoint:Epoch-51.pth')

In [31]:
device='cuda'

In [37]:
for batch in valid_loader:
    mfcc = pad_sequence([sample['mfcc'] for sample in batch], batch_first=True)
    chroma = pad_sequence([sample['chroma'] for sample in batch], batch_first=True)
    rhythm_patterns = pad_sequence([sample['rhythm_patterns'] for sample in batch], batch_first=True)
    start_times = pad_sequence([sample['start_times'] for sample in batch], batch_first=True)
    stop_times = pad_sequence([sample['stop_times'] for sample in batch], batch_first=True)

    print(model(mfcc.to(device), chroma.to(device), rhythm_patterns.to(device)))
    
    print("ACTUAL OUTPUT:")
    
    print(start_times,stop_times)
    
    break


(tensor([[[  9.8808,   4.4377,  79.4566,  88.4239,  67.0505, 122.4196, 158.4926,
           64.3213, 167.2208, 124.7245,  53.0197,  72.3904,  75.0917,  25.4285,
            4.0510]]], device='cuda:0', grad_fn=<ViewBackward0>), tensor([[[ 19.8340,  85.4359,  89.0912,  74.8544,  61.1949, 159.1323, 185.5056,
          173.6344, 163.6078, -41.2310,  90.8854,  66.6665,  32.1955,  34.3917,
           36.9823]]], device='cuda:0', grad_fn=<ViewBackward0>))
ACTUAL OUTPUT:
tensor([[  0.0000,  20.6574,  63.2355,  85.6708, 135.2309, 156.3171, 191.6839,
         241.4335, 283.9351,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000]]) tensor([[ 20.6574,  63.2355,  85.6708, 135.2309, 156.3171, 191.6839, 241.4335,
         283.9351, 290.2846,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000]])


In [ ]:
plt.savefig("results.png")

In [ ]:
print()